# Mount Google drive

In [0]:
%load_ext autoreload
%autoreload 2
  
from google.colab import drive
drive.mount("/content/drive", force_remount=True)


import sys,os
from pathlib import Path


homepath = Path('/content/drive/My Drive/pytorch_Framework_suman/')
sys.path.append(str(homepath))

sys.path.append(os.path.join(homepath,'lib'))
print(sys.path)

!ls

# Initialize device

In [0]:
import torch

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
print(torch.cuda.is_available())

batch_size=1024

# Load Tiny-Imagenet-64 Dataset and Split the dataset (train and validation) into 70% + 30%

In [0]:
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir

In [0]:
from utils.splitDs import *
from lib.dataloader_s12_10 import *

In [0]:
train_dl, test_dl, train_ds, test_ds = load_data()

In [0]:
print('####Train Dataset is : \n\t\n{}\n\n'.format(train_ds))
print('####Validation Dataset is : \n\t\n{}\n\n'.format(test_ds))

print('####Train DL is : \n\t\n{}\n\n'.format(train_dl))
print('####Test DL is : \n\t\n{}\n\n'.format(test_dl))

# Print few images from the train dataset

In [0]:
from matplotlib import pyplot as plt
def show_imgs(img,label):
    print('Image label is : {}'.format(train_ds.classes[label]))
    plt.imshow(img.permute(1, 2, 0))

In [0]:
show_imgs(*test_ds[8])

# Print few images from the train dataloader and test data loader

In [0]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [0]:
def show_batch(dl,numOfImagesToDisplay=10,imgsPerRow=5):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(26, 26))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images[:numOfImagesToDisplay], imgsPerRow).permute(1, 2, 0))
        break

Print few images from Train DataLoader

In [0]:
show_batch(train_dl)

print few Images from Test Data Loader

In [0]:
show_batch(test_dl)

# Test the model what we have written

In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

In [0]:
import torchvision.models as models

model = models.resnet18()

#Finetune Final few layers to adjust for tiny imagenet input
model.avgpool = nn.AdaptiveAvgPool2d(1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 200)
model = model.to(device)

summary(model,(3,64,64))

In [0]:
for images, labels in train_dl:
    images, labels = images.cuda(), labels.cuda()
    print('images.shape:', images.shape)
    out = model(images)
    print('out.shape:', out.shape)
    break

# Implementation of One cycle policy

In [0]:
#!pip install torch-lr-finder -v --global-option="amp"

In [0]:
# LR range test
from lib.torch_lrfinder import *
fro mlib.torch_lrfinder import LRFinder
from torch_lr_finder import LRFinder
import torch.optim as optim

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), 
                      lr=1e-7, 
                      momentum=0.9, 
                      weight_decay=1e-4)

lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
lr_finder.range_test(train_loader=train_dl,
                     val_loader=test_dl, 
                     start_lr=1e-7,
                     end_lr=100, 
                     num_iter=100, 
                     step_mode="exp")
lr_finder.plot(log_lr=True)
lr_finder.reset()

In [0]:
print('Min loss value is : {} \nMin LR value is   : {}'.format(min(lr_finder.history['loss']),format(min(lr_finder.history['lr']),'.10f')))

print('Min loss observed at index : {}'.format(lr_finder.history['loss'].index(min(lr_finder.history['loss']))))
print('so corresponding LR value at that index is--> LRmax : {}'.format(lr_finder.history['lr'][lr_finder.history['loss'].index(min(lr_finder.history['loss']))]))
lrmax = lr_finder.history['lr'][lr_finder.history['loss'].index(min(lr_finder.history['loss']))]
lrmax = float('{:4f}'.format(lrmax))
print(lrmax)

In [0]:
from torch.optim.lr_scheduler import OneCycleLR

optimizer = optim.SGD(model.parameters(), 
                      lr=0.001, 
                      momentum=0.9,
                      weight_decay=1e-4)

scheduler = OneCycleLR(optimizer, 
                       max_lr = lrmax, 
                       total_steps=None,
                       pct_start=0.4, 
                       div_factor=10.0,
                       final_div_factor=100,

                       anneal_strategy='linear', 
                       epochs= 50, 
                       steps_per_epoch=1, 
                       
                       cycle_momentum=False, 
                       base_momentum=0.85, 
                       max_momentum=0.95,

)

In [0]:
trainaccuracies = []
trainlosses = []
vallosses = []
valaccuracies = []
misclassifieditems = []
lr_values = []

In [0]:
from lib import trainTestMethods
for epoch in range(1,2):
    print("\nEPOCH:", epoch,'LR:',optimizer.param_groups[0]['lr'])
    lr_values.append(optimizer.param_groups[0]['lr'])
    trainTestMethods.trainWithAccPlt(network=model, 
                                     trainloader=train_dl, 
                                     device='cuda', 
                                     optimizer=optimizer, 
                                     criterion=criterion, 
                                     trainaccuracies=trainaccuracies, 
                                     trainlosses=trainlosses, 
                                     epoch=epoch)
    
    testloss = trainTestMethods.testWithAccPlt(network=model, 
                                               testloader=test_dl, 
                                               device='cuda', 
                                               criterion=criterion, 
                                               valaccuracies=valaccuracies, 
                                               vallosses=vallosses, 
                                               epoch=epoch)
    scheduler.step()

In [0]:
from lib import trainTestMethods
for epoch in range(2,10):
    print("\nEPOCH:", epoch,'LR:',optimizer.param_groups[0]['lr'])
    lr_values.append(optimizer.param_groups[0]['lr'])
    trainTestMethods.trainWithAccPlt(network=model, 
                                     trainloader=train_dl, 
                                     device='cuda', 
                                     optimizer=optimizer, 
                                     criterion=criterion, 
                                     trainaccuracies=trainaccuracies, 
                                     trainlosses=trainlosses, 
                                     epoch=epoch)
    
    testloss = trainTestMethods.testWithAccPlt(network=model, 
                                               testloader=test_dl, 
                                               device='cuda', 
                                               criterion=criterion, 
                                               valaccuracies=valaccuracies, 
                                               vallosses=vallosses, 
                                               epoch=epoch)
    scheduler.step()

In [0]:
from lib import trainTestMethods
for epoch in range(10,30):
    print("\nEPOCH:", epoch,'LR:',optimizer.param_groups[0]['lr'])
    lr_values.append(optimizer.param_groups[0]['lr'])
    trainTestMethods.trainWithAccPlt(network=model, 
                                     trainloader=train_dl, 
                                     device='cuda', 
                                     optimizer=optimizer, 
                                     criterion=criterion, 
                                     trainaccuracies=trainaccuracies, 
                                     trainlosses=trainlosses, 
                                     epoch=epoch)
    
    testloss = trainTestMethods.testWithAccPlt(network=model, 
                                               testloader=test_dl, 
                                               device='cuda', 
                                               criterion=criterion, 
                                               valaccuracies=valaccuracies, 
                                               vallosses=vallosses, 
                                               epoch=epoch)
    scheduler.step()

# Plot Metrics - Accuracy and Loss

In [0]:
trainTestMethods.plotmetrics(50,
                             trainaccuracies=trainaccuracies, 
                             testaccuracies=valaccuracies, 
                             trainlosses=trainlosses, 
                             testlosses=vallosses, 
                             savefilename="metrics")

In [0]:
plt.plot(lr_values)

In [0]:
model